In [6]:
# import pyads
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

plc_buffer_name = 'Main.profiler_buffer'
plc_numlayers_name = 'Main.FB_NN.nn.num_layers'
plc_numneurons_name = 'Main.FB_NN.nn.layers[[index]].num_neurons'
plc_MeasureFinished_name = 'Main.MeasureFinished'

num_layers = [6, 5, 4, 3]
num_neurons = [500, 400, 300, 200, 100, 10]

In [30]:



measurements = []
with pyads.Connection('127.0.0.1.1.1', pyads.PORT_TC3PLC1) as plc:
    for i in num_layers:
        plc.write_by_name(plc_numlayers_name,i,pyads.PLCTYPE_UINT)
        for j in num_neurons:
            for k in range(i-2):
                plc.write_by_name(plc_numneurons_name.replace('[index]',f'{k+1}'),j,pyads.PLCTYPE_UINT)
            plc.write_by_name(plc_MeasureFinished_name, False ,pyads.PLCTYPE_BOOL)
            while(1):
                plc_MeasureFinished = plc.read_by_name(plc_MeasureFinished_name, pyads.PLCTYPE_BOOL)
                if plc_MeasureFinished:
                    profiler_buffer = plc.read_by_name(plc_buffer_name, pyads.PLCTYPE_DWORD*1000)
                    print(f'num_layers:{i}, num_neurouns:{j}\n')
                    print(profiler_buffer[0:10])
                    for k in profiler_buffer:
                        measurements.append([i,j,k])
                    break

df = pd.DataFrame(measurements, columns=['num_layers', 'num_neurons', 'measurement'])

num_layers:6, num_neurouns:500

[2745, 2761, 2764, 2733, 2783, 2760, 2706, 2747, 2800, 2731]
num_layers:6, num_neurouns:400

[1761, 1755, 1731, 1716, 1705, 1752, 1719, 1744, 1708, 1714]
num_layers:6, num_neurouns:300

[993, 970, 971, 973, 979, 979, 1001, 966, 960, 974]
num_layers:6, num_neurouns:200

[431, 425, 428, 429, 437, 425, 429, 441, 425, 425]
num_layers:6, num_neurouns:100

[108, 108, 108, 109, 108, 108, 109, 108, 108, 108]
num_layers:6, num_neurouns:10

[2, 2, 1, 2, 2, 1, 1, 2, 2, 1]
num_layers:5, num_neurouns:500

[1798, 1838, 1835, 1826, 1858, 1823, 1817, 1831, 1847, 1836]
num_layers:5, num_neurouns:400

[1155, 1157, 1139, 1160, 1148, 1157, 1147, 1156, 1152, 1158]
num_layers:5, num_neurouns:300

[676, 693, 664, 677, 665, 681, 674, 672, 703, 660]
num_layers:5, num_neurouns:200

[294, 292, 292, 292, 291, 291, 291, 292, 293, 292]
num_layers:5, num_neurouns:100

[76, 76, 76, 77, 76, 76, 76, 76, 76, 76]
num_layers:5, num_neurouns:10

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
num_layers:4, n

In [3]:
df.to_pickle('exe_time_measurement.pkl')

NameError: name 'df' is not defined

In [7]:
df = pd.read_pickle('exe_time_measurement.pkl')

In [8]:
color_map = {6: 'rgb(255,0,0)', 5: 'rgb(0,255,0)', 4: 'rgb(0,0,255)', 3: 'rgb(255,255,0)'}

fig = px.box(df, x="num_neurons", y="measurement", color="num_layers",boxmode = 'overlay', color_discrete_map=color_map)
fig.update_layout(xaxis_title="Number of Neurons in each layer", yaxis_title="Execution Time [us]", legend_title = 'Number of Layers')
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = num_neurons
    )
)

for i in num_layers:
    mean = []
    for j in num_neurons:
        mean.append(df.loc[(df['num_layers'] == i) & (df['num_neurons'] == j)]['measurement'].mean())
    fig.add_trace(go.Scatter(x=num_neurons, y=mean, showlegend=False,  marker_color=color_map[i] ))
fig.show()

c:\Users\ac143081\AppData\Local\Programs\Python\Python312\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [22]:
# to calculate the mean,min, max of different number of neurons and different number of layers
# then save the result into a .dat file, number of neurons as x, number of layers as y, mean, max - mean, mean - min as value

# create folder 
import os
folder = 'exe_time_measurement'
if not os.path.exists(folder):
    os.makedirs(folder)

for i in num_layers:
    mean = []
    max = []
    min = []

    mean.append([])
    max.append([])
    min.append([])
    for j in num_neurons:
        mean[-1].append(df.loc[(df['num_layers'] == i) & (df['num_neurons'] == j)]['measurement'].mean())
        max[-1].append(df.loc[(df['num_layers'] == i) & (df['num_neurons'] == j)]['measurement'].max())
        min[-1].append(df.loc[(df['num_layers'] == i) & (df['num_neurons'] == j)]['measurement'].min()) 
    filename = os.path.join(folder, f"num_layers_{i}.dat")
    with open(filename, 'w') as f:
        f.write('x y ey+ ey-\n')
        for j in range(len(num_neurons)):
            f.write(f'{num_neurons[j]} {mean[0][j]} {max[0][j]-mean[0][j]} {mean[0][j]-min[0][j]}\n')

            

